# Jugando con nested sampling

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
plt.rcParams['figure.dpi'] = 120
rng = np.random.default_rng(42)

In [ ]:
def rosenbrock(x, y, a, b):
    """
    Typical concave function used to test optimization
    algorithms
    """
    return (a - x) ** 2 + b * (y - x ** 2) ** 2

Para empezar, vamos a usar una verosimilitud de Rosenbrock

In [ ]:
def likelihood(x, y):
    return 1 / (1 +  rosenbrock(x, y, a=0.5, b=100))

xcoord1d = np.linspace(-2, 2, 250)
ycoord1d = np.linspace(-1, 3, 250)
x, y = np.meshgrid(xcoord1d, ycoord1d)
a = 0.5
b = 100
L = likelihood(x, y)
dx_grid = xcoord1d[1] - xcoord1d[0]
dy_grid = ycoord1d[1] - ycoord1d[0]

true_evidence = np.sum(L) * dx_grid * dy_grid
print(f'True evidence = {true_evidence:.2}')

fig, ax = plt.subplots()
extent = (xcoord1d[0], xcoord1d[-1], ycoord1d[0], ycoord1d[-1])
im_ = ax.imshow(
    L,
    origin='lower',
    norm=LogNorm(),
    extent=extent,
)
ax.contour(
    L,
    levels=np.array([0.2, 0.5, 0.9]) * L.max(), 
    extent=extent,
    colors='white'
)

fig.colorbar(im_, shrink=0.82)

ax.axvline(a, color='red')
ax.axhline(a ** 2, color='red')
ax.plot([a], [a ** 2], marker='s', ls='', color='red', mec='k')
ax.set_xlabel('x')
ax.set_ylabel('y');

## Inicialización
El primer paso es hacer un muestreo de los priors con 100 puntos vivos y evaluar la verosimilitud $L$ en cada punto. En este caso voy a usar priors uniformes entre $-2<x<2$ y $-1<y<3$.

In [ ]:
Nlive = 100
x_live = rng.uniform(low=-2, high=2, size=Nlive)
y_live = rng.uniform(low=-1, high=3, size=Nlive)

live_likelihoods = likelihood(x_live, y_live)

fig, ax = plt.subplots()
ax.imshow(L, extent=extent, origin='lower', norm=LogNorm())
ax.scatter(x_live, y_live, marker='o', color='red');

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.sort(live_likelihoods), ls='', marker='o')
ax.set_yscale('log')
ax.set_ylabel('Likelihood')
ax.set_xlabel('Sorted live point index')
ax.annotate('Lowest likelihood point', (0, live_likelihoods.min()*1.1), xytext=(0, 1e-2), arrowprops=dict(width=1, color='k'));


## Reducción
Ahora tengo que eliminar el punto con menor verosimilitud $L_0$. Cada punto vivo representa aproximadapente $1/N$ del volumen total. Por lo tanto, eliminar este punto reduce el volumen por un diferencial de aproximadamente $\delta V \approx 1/N$. Para ser más precisos, $\delta V$ es una variable aleatoria que se distribuye según $\mathrm{Beta}(1, N)$. Entonces, para estimar $\delta V$ uno puede tomar una muestra de dicha distribución, o bien, usar la media geométrica $1 - \exp(-1/N)$ o la media aritmética $1/(N+1)$. En la práctica, da un poco lo mismo.

In [ ]:
index_L0 = np.argmin(live_likelihoods)
dead_points = list()
dead_points.append((x_live[index_L0], y_live[index_L0],  live_likelihoods[index_L0]))

## Muestreo de prior con verosimilitud restringida
Una vez que eliminamos nuestro punto de más baja verosimilitud, tenemos que reemplazarlo por un punto cuya verosimilitud sea mayor. En inglés esto se llama _likelihood-restricted prior sampling_ o LPRS y es el paso más importante de NS. Sin embargo, hacer esto de manera eficiente no es tan sencillo. La estrategia de fuerza bruta es volver a muestrear aleatoriamente el prior varias veces hasta que salga un punto con $L>L_0$. Para hacer esto un poquito más eficiente, podemos restringir el espacio prior al rectángulo más pequeño que contenga todos los puntos vivos

In [ ]:
L_new = 0
while L_new <= live_likelihoods[index_L0]:
    x_new = rng.uniform(low=x_live.min(), high=x_live.max())
    y_new = rng.uniform(low=y_live.min(), high=y_live.max())
    L_new = likelihood(x_new, y_new)

x_live[index_L0] = x_new
y_live[index_L0] = y_new
live_likelihoods[index_L0] = L_new

## Iteración
Veamos hasta dónde llegamos despúes de 1000 iteraciones

In [ ]:
for _ in range(1_000):
    index_L0 = np.argmin(live_likelihoods)
    dead_points.append((x_live[index_L0], y_live[index_L0],  live_likelihoods[index_L0]))
    L_new = 0
    while L_new < live_likelihoods[index_L0]:
        x_new = rng.uniform(low=x_live.min(), high=x_live.max())
        y_new = rng.uniform(low=y_live.min(), high=y_live.max())
        L_new = inverse_rosenbrock(x_new, y_new, a, b)

    x_live[index_L0] = x_new
    y_live[index_L0] = y_new
    live_likelihoods[index_L0] = L_new
    

In [ ]:
dead_points_array = np.array(dead_points).T
fig, ax = plt.subplots()
ax.plot(dead_points_array[2])
ax.set_xlabel('Número de iteración')
ax.set_ylabel(r'$L_0$')
ax.set_yscale('log');

## Integración

In [ ]:
volume_array = (1 - 1 / Nlive)  ** np.arange(dead_points_array.shape[1]) * (xcoord1d[-1] - xcoord1d[0]) * (ycoord1d[-1] - ycoord1d[0])
dV_array = volume_array * 1 / Nlive
weights = dead_points_array[2] * dV_array
fig, ax = plt.subplots()
ax.plot(-np.log(volume_array), weights*1e4)
ax.set_xlabel(r'-log(Volume)')
ax.set_ylabel(r'Weights $(\Delta V_i \times L_i$)')

In [ ]:
fig, ax = plt.subplots()
ax.plot(-np.log(volume_array), np.cumsum(weights))
ax.set_xlabel('-log(Volume)')
ax.set_ylabel('Evidence')

In [ ]:
plt.scatter(dead_points_array[0], dead_points_array[1], c=np.cumsum(weights))

Sugerencias Nico para avanzar:
 - Usar github (coordinar con Gabriel)
 - Comparar con integración por grilla
 - Entender de dónde sale la distribución Beta
 - Separar los artículos en una serie (ns básico, algoritmos de LRPS, estado del arte, etc.)